# Confusion Detection - Proposed Method with GPT-4o

detection accuracy = 82.46%

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import sys
import base64
import requests
from PIL import Image
import io
import json
import uvicorn
from fastapi import FastAPI, File, UploadFile, Form
from fastapi.responses import FileResponse, StreamingResponse, JSONResponse
from PIL import Image, ImageOps, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import numpy as np
import io
import subprocess
import torch
import json
import os
import warnings
import numpy as np
import matplotlib.pyplot as plt
import requests
from PIL import Image
from io import BytesIO
from lang_sam import LangSAM
import torch
import cv2
import base64
import openai
from openai import OpenAI
import os
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import queue
import subprocess

In [2]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# account 1: google login
os.environ["OPENAI_API_KEY"] = "..."
api_key = '...'

question = """Here are two images. The first one is a raw image, the second one is an augmented image, created by adding some virtual content to the space.
Please answer the following questions:
1.What is the virtual content in the augmented image?
2.What "key object" is interacting with the virtual content? Avoid general terms like “table surface” or “environment”.
3.Is the virtual content accurately aligned to object, without significant gap? Answer yes or no, then explain why.
4.Does the virtual content have a relatively high quality of texture? Answer yes or no, then explain why.
5.Do you think the interaction will make users to believe the "key object" has some false functionality or features it doesn’t have, 
or lost some true functionality it actually has? Think creatively, don't say "no" easily, only say no if the combination has no specific semantic relation. Answer yes or no, then explain why.
6.If you answered “yes” in all question 3, 4 and 5, you must say “yes”. Otherwise you say “no”."""


headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

result_csv = 'confusion_gpt.csv'
response_txt = 'response_gpt.txt'

In [34]:
def check_last_occurrence(input_string):
    # Split the string into words
    words = input_string.lower().split()

    # Initialize variables to store the last positions of "yes" and "no"
    last_yes = -1
    last_no = -1

    # Iterate through the words and track the positions of "yes" and "no"
    for i, word in enumerate(words):
        if word == "yes" or word == "Yes" or word == "Yes." or word == "yes." or word == "Yes," or word == "yes," or word == "**Yes**" or word == "**yes**":
            last_yes = i
        elif word == "no" or word == "No" or word == "No." or word == "no." or word == "No," or word == "no," or word == "**No**" or word == "**no**":
            last_no = i

    # Compare the positions to determine which appeared last
    if last_yes > last_no:
        return 1
    elif last_no > last_yes:
        return 0
    else:
        return 'N/A'  # In case neither "yes" nor "no" are found

In [52]:
# read the csv file
df = pd.read_csv('confusion_dataset.csv')

ar_folder = '.../confusion data/ar'
raw_folder = '.../confusion data/raw'

# # clear the txt
# with open(response_txt, 'w') as f:
#     f.write('')
    
start_point = 0

for i in range(start_point, len(df)):
    ar_img_name = ar_folder + '/' + str(i+1) + '_ar.png'
    raw_img_name = raw_folder + '/' + str(i+1) + '_raw.png'
    
    ar_img = encode_image(ar_img_name)
    raw_img = encode_image(raw_img_name)
    
    label = df.loc[i, 'confusion']
    
    payload = {
        # ! This is the model for GPT-4 Vision
        # "model": "gpt-4-vision-preview",
        "model": "gpt-4o",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": question
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{raw_img}",
                            "detail": "high"
                        }
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{ar_img}",
                            "detail": "high"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 600
    }
    
    bad_response = True
    bad_response_list = ['sorry']
    
    while bad_response:
    
        # trial += 1
        # print(f'Trial {trial}')

        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
        
        # ! if there's no 'error' in the keys:
        if 'error' in response.json().keys():
            print('Error!')
            print(response.json())
            continue
        text_response = response.json()['choices'][0]['message']['content']
        
        # remove " and ' in the response
        text_response = text_response.replace('\"', '')
        text_response = text_response.replace("\'", '')
        
        if any(bad in text_response for bad in bad_response_list) == False:
            bad_response = False
            break
        
    # remove \n, , in the response
    text_response = text_response.replace('\n', ' ')
    text_response = text_response.replace(',', ' ')
    
    # write the response to a txt file
    with open(response_txt, 'a') as f:
        f.write(f'File {i+1}: {text_response}\n')
    
    
    # check "yes" or "no" that appear in the last
    prediction = check_last_occurrence(text_response)
    
    
    
    # claculate score
    score = 0
    if prediction == 'N/A':
        score = 0
    elif prediction == label:
        score = 1
    else:
        score = 0
    
        
        
    # write the response to another csv file
    with open(result_csv, 'a') as f:
        f.write(f'{i+1},{label},{prediction},{score}\n')
        
    # if i == start_point + 7:
    #     break
    
print('Done!')
# read the result csv and calculate the score
df_result = pd.read_csv(result_csv, header=None)
df_result[3][1:] = df_result[3][1:].astype(int)
total_score = df_result[3][1:].sum()
print(f'Acc: {total_score}/{len(df_result)-1}')


Done!
Acc: 94/114


/tmp/ipykernel_644585/2194542101.py:114: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_result[3][1:] = df_result[3][1:].astype(int)


In [54]:
df_result = pd.read_csv(result_csv, header=None)
df_result[3][1:] = df_result[3][1:].astype(int)
total_score = df_result[3][1:].sum()
print(f'Acc: {total_score}/{len(df_result)-1} = {total_score/(len(df_result)-1)}')


Acc: 94/114 = 0.8245614035087719


/tmp/ipykernel_644585/2392947878.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_result[3][1:] = df_result[3][1:].astype(int)
